<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/Fundus-eye-disease-detection/custom_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (
    Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
)
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Constants
IMG_SIZE = 224
CATEGORIES = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']
BATCH_SIZE = 32
EPOCHS = 25
DATA_DIR = '/content/drive/MyDrive/data set/dataset'

# Data Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split for training and validation
)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Calculate class weights to address imbalance
y_train = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_tensor = tf.constant([class_weights[i] for i in range(len(CATEGORIES))])

# Define a wrapper for the dual input generator
def dual_input_generator(generator):
    for x, y in generator:
        yield ([x, x], y)

# Create datasets from the generators with output_signature
train_dual_dataset = tf.data.Dataset.from_generator(
    lambda: dual_input_generator(train_generator),
    output_signature=(
        (tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32)),
        tf.TensorSpec(shape=(None, len(CATEGORIES)), dtype=tf.float32)
    )
)

val_dual_dataset = tf.data.Dataset.from_generator(
    lambda: dual_input_generator(val_generator),
    output_signature=(
        (tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32)),
        tf.TensorSpec(shape=(None, len(CATEGORIES)), dtype=tf.float32)
    )
)

# Model: Combine VGG16 and ResNet50 with custom layers
# Load pre-trained VGG16 and ResNet50
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the pre-trained layers
for layer in vgg_base.layers:
    layer.trainable = False

for layer in resnet_base.layers:
    layer.trainable = False

# Combine features from both models
vgg_features = vgg_base.output
resnet_features = resnet_base.output

# Apply Global Average Pooling to each
vgg_gap = GlobalAveragePooling2D()(vgg_features)
resnet_gap = GlobalAveragePooling2D()(resnet_features)

# Concatenate the outputs
merged_features = tf.keras.layers.concatenate([vgg_gap, resnet_gap])

# Add custom layers on top
x = Dense(256, activation='relu')(merged_features)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(len(CATEGORIES), activation='softmax')(x)

# Build the model
model = Model(inputs=[vgg_base.input, resnet_base.input], outputs=output)

# Define custom loss function with class weights
from tensorflow.keras.losses import CategoricalCrossentropy

def weighted_categorical_crossentropy(y_true, y_pred):
    # Cast class weights to float32 to match y_true dtype
    weights = tf.cast(class_weight_tensor, dtype=tf.float32)
    weighted = tf.reduce_sum(weights * y_true, axis=-1)
    unweighted_loss = CategoricalCrossentropy()(y_true, y_pred)
    return unweighted_loss * weighted

# Compile the model with the custom loss
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss=weighted_categorical_crossentropy,
    metrics=['accuracy']
)

# Model summary
model.summary()

# Callbacks
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.1, patience=5, verbose=1, min_lr=1e-6
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=7, restore_best_weights=True, verbose=1
    )
]

# Train the model
history = model.fit(
    train_dual_dataset,
    epochs=EPOCHS,
    validation_data=val_dual_dataset,
    callbacks=callbacks,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

# Evaluate the model
val_loss, val_acc = model.evaluate(val_dual_dataset, steps=len(val_generator))
print(f"Validation Accuracy: {val_acc:.4f}")

# Get predictions on the validation set
y_pred_probs = model.predict(val_dual_dataset, steps=len(val_generator))
y_pred = np.argmax(y_pred_probs, axis=1)

# Get true labels for the validation set
y_true = val_generator.classes

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy on Validation Set: {accuracy:.4f}")

# Save the model
model.save('/content/custom_eye_disease_model.h5')
print("Model saved successfully.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 3384 images belonging to 4 classes.
Found 844 images belonging to 4 classes.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 38,960,068 (148.62 MB)

 Trainable params: 657,156 (2.51 MB)

 Non-trainable params: 38,302,912 (146.11 MB)

Epoch 1/25


InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was ((tf.float32, tf.float32), tf.float32), but the yielded element was ([array([[[[  2.        ,   2.        ,   2.        ],
         [  2.        ,   2.        ,   2.        ],
         [  2.        ,   2.        ,   2.        ],
         ...,
         [  3.        ,   3.        ,   3.        ],
         [  3.9980416 ,   3.9980416 ,   3.9980416 ],
         [  1.847847  ,   1.847847  ,   1.847847  ]],

        [[  2.        ,   2.        ,   2.        ],
         [  2.        ,   2.        ,   2.        ],
         [  2.        ,   2.        ,   2.        ],
         ...,
         [  3.0510547 ,   3.0510547 ,   3.0510547 ],
         [  3.5896678 ,   3.5896678 ,   3.5896678 ],
         [  1.6407225 ,   1.6407225 ,   1.6407225 ]],

        [[  2.   